In [10]:
!pip install pdfplumber pandas --quiet


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
import re
from pathlib import Path
import pandas as pd
import pdfplumber

In [25]:
def extract_year_week_from_filename(file_path: str):
    """
    Extrai ano e semana a partir do padrão ES2548:
    ES2548 -> 25 = ano (2025), 48 = semana do inventário
    """
    name = Path(file_path).stem
    m = re.search(r"ES(\d{2})(\d{2})", name)
    if not m:
        return None, None

    ano_codigo = int(m.group(1))
    semana_codigo = int(m.group(2))

    ano = 2000 + ano_codigo
    semana = semana_codigo
    return ano, semana


def ler_tabela_estufas(file_path: str) -> pd.DataFrame:
    """
    Procura, em todas as páginas do PDF, a tabela cujo cabeçalho
    contém 'Bloco' E 'Área Total' (ou 'Area Total').
    Só essa tabela é considerada o inventário principal.
    """
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            tables = page.extract_tables()
            for t in tables:
                df_raw = pd.DataFrame(t)

                # percorre as linhas procurando o cabeçalho
                for i in range(len(df_raw)):
                    row = df_raw.iloc[i].astype(str).str.strip()
                    values = set(row.values)

                    tem_bloco = any(v.lower() == "bloco" for v in values)
                    tem_area_total = any(
                        v.lower() in ("área total", "area total") for v in values
                    )

                    if tem_bloco and tem_area_total:
                        # essa é a linha de cabeçalho
                        header = row.tolist()
                        df_data = df_raw.iloc[i + 1 :].reset_index(drop=True)
                        df_data.columns = header
                        return df_data

    print(f"Tabela de inventário (com Área Total) não encontrada em {file_path}")
    return pd.DataFrame()


def limpar_e_transformar(df: pd.DataFrame, ano: int, semana_inv: int) -> pd.DataFrame:
    """
    Limpa e cria as colunas no formato desejado.
    """

    # normaliza nomes de colunas
    norm_map = {}
    for col in df.columns:
        norm = (
            str(col)
            .strip()
            .lower()
            .replace("\n", " ")
        )
        norm_map[col] = norm

    rename_map = {
        "bloco": "bloco",
        "naves": "naves",
        "nº naves": "n_naves",
        "no naves": "n_naves",
        "n naves": "n_naves",
        "área/nave": "area_nave",
        "area/nave": "area_nave",
        "área total": "area_total",
        "area total": "area_total",
        "cultura": "cultura",
        "idade": "idade",
        "semana do plantio": "semana_plantio",
        "data do plantio": "data_plantio",
    }

    # aplica rename baseado no nome normalizado
    rename_dict = {}
    for original, norm in norm_map.items():
        if norm in rename_map:
            rename_dict[original] = rename_map[norm]

    df = df.rename(columns=rename_dict)

    # mantém só as colunas de interesse, se existirem
    cols_interesse = [
        "bloco",
        "naves",
        "n_naves",
        "area_nave",
        "area_total",
        "cultura",
        "idade",
        "semana_plantio",
        "data_plantio",
    ]
    df = df[[c for c in cols_interesse if c in df.columns]].copy()

    # 🔥 FILTRO EXTRA: dropar linhas sem Naves (elimina as tabelas de cultura)
    if "naves" in df.columns:
        df["naves"] = df["naves"].astype(str).str.strip()
        df = df[df["naves"] != ""]
        df = df[~df["naves"].isna()]

    # bloco -> inteiro
    df["bloco"] = pd.to_numeric(df["bloco"], errors="coerce")
    df = df.dropna(subset=["bloco"])
    df["bloco"] = df["bloco"].astype(int)

    # "Bloco 1", "Bloco 2", ...
    df["bloco_modelado"] = "Bloco " + df["bloco"].astype(str)

    # nº naves
    if "n_naves" in df.columns:
        df["n_naves"] = pd.to_numeric(df["n_naves"], errors="coerce")

    # Área/Nave e Área Total
    for col in ["area_nave", "area_total"]:
        if col in df.columns:
            df[col] = (
                df[col].astype(str)
                .str.replace(".", "", regex=False)
                .str.replace(",", ".", regex=False)
            )
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # idade e semana do plantio
    for col in ["idade", "semana_plantio"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # data do plantio
    if "data_plantio" in df.columns:
        df["data_plantio_str"] = df["data_plantio"].astype(str)
        df["data_plantio"] = pd.to_datetime(
            df["data_plantio_str"].str.extract(r"(\d{2}/\d{2}/\d{4})", expand=False),
            format="%d/%m/%Y",
            errors="coerce",
        )

    # ano e semana do inventário
    df["ano"] = ano
    df["semana_inventario"] = semana_inv

    # ordena colunas
    col_order = [
        "bloco_modelado",
        "bloco",
        "naves",
        "n_naves",
        "area_nave",
        "area_total",
        "cultura",
        "idade",
        "semana_plantio",
        "data_plantio",
        "ano",
        "semana_inventario",
    ]
    df = df[[c for c in col_order if c in df.columns]]

    return df


def processar_inventario_pdf(file_path: str) -> pd.DataFrame:
    """
    Processa UM PDF:
      - extrai ano/semana do nome
      - encontra a tabela com 'Área Total'
      - limpa e modela as colunas
    """
    ano, semana_inv = extract_year_week_from_filename(file_path)
    if ano is None:
        print(f"⚠ Não consegui extrair ano/semana de {file_path}")
        return pd.DataFrame()

    base = ler_tabela_estufas(file_path)
    if base.empty:
        return pd.DataFrame()

    df = limpar_e_transformar(base, ano, semana_inv)
    df["arquivo_origem"] = Path(file_path).name
    return df


def processar_pasta_inventario(dir_path: str) -> pd.DataFrame:
    """
    Varre TODOS os PDFs da pasta e concatena num único DataFrame.
    Depois remove linhas sem Naves (para eliminar tabelas de cultura).
    """
    pasta = Path(dir_path)
    pdfs = list(pasta.glob("*.pdf"))

    print(f"Encontrados {len(pdfs)} PDFs na pasta {pasta}.")

    dfs = []
    for pdf in pdfs:
        print(f"Processando: {pdf.name}")
        df_tmp = processar_inventario_pdf(str(pdf))
        if not df_tmp.empty:
            dfs.append(df_tmp)

    if not dfs:
        print("⚠ Nenhum dado processado.")
        return pd.DataFrame()

    df_final = pd.concat(dfs, ignore_index=True)

    # 🔥 LIMPEZA FINAL: remover linhas sem Naves
    if "naves" in df_final.columns:
        df_final["naves"] = df_final["naves"].astype(str).str.strip()
        df_final = df_final[
            (df_final["naves"].notna()) &
            (df_final["naves"] != "") &
            (df_final["naves"].str.lower() != "none")
        ]

    print("✔ Processamento concluído após limpeza final.")
    return df_final

In [26]:
df_final = processar_pasta_inventario(
    r"C:\Users\cauai.Capozzoli\Desktop\DB\estufas\inventario"
)

df_final

Encontrados 2 PDFs na pasta C:\Users\cauai.Capozzoli\Desktop\DB\estufas\inventario.
Processando: ES2548 Inventário Estufas - Semana 48 2025.pdf
Processando: ES2549 Inventário Estufas - Semana 49 2025.pdf
✔ Processamento concluído após limpeza final.


,bloco_modelado,bloco,naves,n_naves,area_nave,area_total,cultura,idade,semana_plantio,data_plantio,ano,semana_inventario,arquivo_origem
0,Bloco 1,1,1 a 23,14.0,0.057,0.798,Flores,NaN,NaN,NaT,2025,48,ES2548 Inventário Estufas - Semana 48 2025.pdf
1,Bloco 2,2,1 a 19,19.0,0.057,1.083,Flores,NaN,NaN,NaT,2025,48,ES2548 Inventário Estufas - Semana 48 2025.pdf
2,Bloco 3,3,1 a 19,19.0,0.057,1.083,Operações Campo Aberto e Café,NaN,NaN,NaT,2025,48,ES2548 Inventário Estufas - Semana 48 2025.pdf
3,Bloco 4,4,1 a 8,8.0,0.057,0.456,Em limpeza.,NaN,NaN,NaT,2025,48,ES2548 Inventário Estufas - Semana 48 2025.pdf
4,Bloco 4,4,10 a 17,8.0,0.057,0.456,Operações Campo Aberto e Café,NaN,NaN,NaT,2025,48,ES2548 Inventário Estufas - Semana 48 2025.pdf
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,Bloco 21,21,1 a 20,20.0,0.060,1.200,Em limpeza.,NaN,NaN,NaT,2025,49,ES2549 Inventário Estufas - Semana 49 2025.pdf
70,Bloco 22,22,1 a 20,20.0,0.060,1.200,Em preparo de Solo (Tomate),NaN,NaN,NaT,2025,49,ES2549 Inventário Estufas - Semana 49 2025.pdf
71,Bloco 23,23,1 a 20,20.0,0.060,1.200,Tomate,9.0,40.0,2025-09-30,2025,49,ES2549 Inventário Estufas - Semana 49 2025.pdf
72,Bloco 24,24,1 a 20,20.0,0.060,1.200,Flores,NaN,NaN,NaT,2025,49,ES2549 Inventário Estufas - Semana 49 2025.pdf


In [27]:
df_final.to_excel("inventario_estufas_silver.xlsx", index=False)